# 01_import dataset

## 01-1_import data

In [1]:
import os
import json
import pandas as pd

#https://drive.google.com/drive/folders/18qV82fNY3IIWu3BRoGqm_LNgJzE8Akbr?usp=drive_link
#base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"
base_dir= '/Users/andyw.p.chen/Documents/Project/datasets'
#base_dir=  "c:\Users\user\Downloads\datasets"

def load_json_to_df(filename: str) -> pd.DataFrame:
    file_path = os.path.join(base_dir, filename)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 如果是 { "target": {id: value, ...} }
    if isinstance(data, dict) and len(data) == 1 and isinstance(next(iter(data.values())), dict):
        key, inner = next(iter(data.items()))
        return pd.DataFrame(list(inner.items()), columns=["id", key])

    # dict of scalar
    if isinstance(data, dict):
        return pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])

    # list of dict
    elif isinstance(data, list):
        return pd.DataFrame(data)

    else:
        raise ValueError(f"Unsupported JSON structure in {filename}: {type(data)}")


def load_csv_to_df(filename: str) -> pd.DataFrame:
    """讀取 CSV 並轉為 DataFrame。"""
    return pd.read_csv(os.path.join(base_dir, filename))

# JSON 資料
##mcc_codes_df = load_json_to_df("mcc_codes.json")
train_fraud_labels_df = load_json_to_df("train_fraud_labels.json")

# CSV 資料
cards_df = load_csv_to_df("cards_data.csv")
transactions_df = load_csv_to_df("transactions_data.csv")
users_df = load_csv_to_df("users_data.csv")

# 簡單檢查
#print(mcc_codes_df.head())
#print(train_fraud_labels_df.head())
#print(cards_df.head())
#print(transactions_df.head())
#print(users_df.apthead())


## 01-2_rename variable in each data set

In [2]:
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'id': 'transactions_id'})
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'target': 'is_fraud'})

cards_df = cards_df.rename(columns={'id':'card_id'})

users_df = users_df.rename(columns={'id':'client_id'})

transactions_df = transactions_df.rename(columns={'mcc': 'mcc_code'})
transactions_df = transactions_df.rename(columns={'id': 'transaction_id'})




## 01-3_變數型態統一及缺失值處理

In [3]:
def add_missing_flags(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    在 DataFrame 中對指定欄位建立 missing flag 欄位
    flag=1 表示缺失值，flag=0 表示非缺失值
    
    參數
    ----
    df : pd.DataFrame
        輸入的資料框
    cols : list
        要檢查的欄位名稱清單
    
    回傳
    ----
    pd.DataFrame : 新的資料框 (含新增的 flag 欄位)
    """
    for col in cols:
        df[f"{col}_missing_flag"] = df[col].isna().astype(int)
    return df

transactions_df = add_missing_flags(transactions_df, ["merchant_state", "zip", "errors"])

In [4]:
##train_fraud_labels_df##
train_fraud_labels_df["is_fraud"]=train_fraud_labels_df["is_fraud"].astype("category") 
train_fraud_labels_df["transactions_id"]=train_fraud_labels_df["transactions_id"].astype(int) #合併資料需要

##cards_df##
cards_df["card_brand"]=cards_df["card_brand"].astype("category") 
cards_df["card_type"]=cards_df["card_type"].astype("category")
#####不要load這行 cards_df["expires"]=pd.to_datetime(cards_df["expires"], format="%m/%Y")
cards_df["expires"] = pd.to_datetime(cards_df["expires"], format="%m/%Y").dt.to_period("M")
cards_df["has_chip"]=cards_df["has_chip"].astype("category")

cards_df['credit_limit'] = cards_df['credit_limit'].replace(r'[\$,]', '', regex=True).astype(int)
#####不要load這行 cards_df["acct_open_date"]=pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y")
cards_df["acct_open_date"] = pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y").dt.to_period("M")
#####不要load這行 cards_df["year_pin_last_changed"]=pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y")
cards_df["year_pin_last_changed"] = pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y").dt.to_period("Y")
cards_df["card_on_dark_web"]=cards_df["card_on_dark_web"].astype("category") 

##users_df##
users_df["birth_year"] = pd.to_datetime(users_df["birth_year"], format="%Y").dt.to_period("Y")
users_df["birth_month"] = pd.to_datetime(users_df["birth_month"], format="%m").dt.to_period("M")
users_df["gender"]=users_df["gender"].astype("category") 
users_df['per_capita_income'] = users_df['per_capita_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['yearly_income'] = users_df['yearly_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['total_debt'] = users_df['total_debt'].replace(r'[\$,]', '', regex=True).astype(int)

##transactions_df##
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
#浮點數轉整數原因確定？
transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float).astype(int)
##負數取log調成1
#transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float)

transactions_df["use_chip"]=transactions_df["use_chip"].astype("category") 

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'merchant_state'
] = 'online'

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'zip'
] = 20000 #原本是-1
## 我沒有全部改，這樣完之後仍有89006筆Missing，剩下都是在國外
transactions_df['zip'] = transactions_df['zip'].fillna(10000) #原本是-999
transactions_df["zip"]=transactions_df["zip"].astype("int64")

transactions_df['errors'] = transactions_df['errors'].astype('category')
transactions_df['errors'] = transactions_df['errors'].cat.add_categories('No_error').fillna('No_error')



In [5]:
#cars one hot encoding
##統一類別變數轉dummy variable(要注意共線性問題，應刪掉其中之一)

#card_type 原始種類：Debit_57%, Credit_33%, Debit(Prepaid)_9%
#card_brand 原始種類：MasterCard_52%, Visa_38%, Amex_7%, Discovery_3%
#has_chip 原始種類：Yes_89%, No_11%
#card_on_dark_web 原始種類：No_0%
cols_to_encode = ['card_type', 'card_brand', 'has_chip']
cards_df[cols_to_encode] = cards_df[cols_to_encode].astype('category')
dummies_cards = pd.get_dummies(
    cards_df[cols_to_encode], 
    prefix=cols_to_encode, 
    dtype='uint8'
    )
cards_df = pd.concat([cards_df, dummies_cards], axis=1)

#use_chip 原始種類：Swiped_52%, Chipe_36%, Online_12%
dummies_use = pd.get_dummies(transactions_df['use_chip'], prefix='use_chip', dtype='uint8')
transactions_df = pd.concat([transactions_df, dummies_use], axis=1)

#gender 原始種類：Female_51%, Male_49%
dummies_gender = pd.get_dummies(users_df['gender'], prefix='gender', dtype='uint8')
users_df = pd.concat([users_df, dummies_gender], axis=1)


cards_df.drop(columns=["has_chip_NO","has_chip"], inplace=True)
transactions_df.drop(columns=["use_chip"], inplace=True)
users_df.drop(columns=["gender_Female"], inplace=True)

# 02_資料整併成一張dataframe

## 02-1_資料整併

In [6]:
#transactions_df.loc[transactions_df["transaction_id"] == 10649266] #transaction_id vs id

#原始資料筆數：13305915
### transactions_df+train_fraud_labels_df      left 會有4390952 missing values
merged = pd.merge(transactions_df, train_fraud_labels_df, left_on="transaction_id", right_on="transactions_id", how="outer")
### transactions_df train_fraud_labels_df(8914963) + users_df 對過去不會有missing values
merged = pd.merge(merged,users_df , left_on="client_id", right_on="client_id", how="left")
### transactions_df train_fraud_labels_df users_df + cards_df 對過去不會有missing values
merged = pd.merge(merged,cards_df , left_on="card_id", right_on="card_id", how="left")

#刪掉重複的columns
merged.drop(columns=["transactions_id"], inplace=True)
merged.drop(columns=["client_id_y"], inplace=True)

## 合併完之後最後處理is_fraud(原會有missing values問題)
merged["is_fraud"] = merged["is_fraud"].astype(str)
merged.loc[merged['is_fraud'].str.lower() == 'no','is_fraud'] = '0'
merged.loc[merged['is_fraud'].str.lower() == 'yes','is_fraud'] = '1'
merged["is_fraud"] = pd.to_numeric(merged["is_fraud"], errors="coerce").astype("Int64")

merged = add_missing_flags(merged, ["is_fraud"])

#merged.to_csv("merged.csv", index=False)

# 先刪除不需要的DataFrame以節省記憶體
del transactions_df, users_df, cards_df, train_fraud_labels_df, cols_to_encode, dummies_cards, dummies_use, dummies_gender

In [7]:
backup_merged = merged.copy()
#merged = backup_merged.copy()

# 04_RFM features engineering model

## 04-1_資料進行變數轉換以求模型配飾更佳表現

In [8]:
##有出事再趕快回復原狀
mask = (merged["date"] >= "2018-01-01 00:00:00") & (merged["date"] <= "2020-12-31 23:59:59")
merged = merged[mask]
#merged = backup_merged.copy()

In [9]:
import numpy as np
import pandas as pd

# 確保日期是 datetime 並排序
merged['date'] = pd.to_datetime(merged['date'])
merged = merged.sort_values(by=['client_id_x', 'date']).reset_index(drop=True)

# --- RecencyInterval ---
merged['RecencyInterval'] = merged.groupby('client_id_x')['date'].diff().dt.total_seconds().fillna(0)/60

# --- TxnFrequency for multiple windows (向量化滑動窗口) ---
window_days = [7, 30, 60, 90]
for w in window_days:
    merged[f'TxnFrequency_{w}d'] = 0

def compute_freq_vectorized(dates, windows):
    """向量化計算每筆交易在每個 window 內的交易數"""
    n = len(dates)
    dates_int = dates.values.astype('datetime64[D]').astype(int)
    res = {w: np.zeros(n, dtype=int) for w in windows}
    for w in windows:
        left = 0
        counts = np.zeros(n, dtype=int)
        for right in range(n):
            while dates_int[right] - dates_int[left] > w:
                left += 1
            counts[right] = right - left + 1
        res[w] = counts
    return res

# 分組計算
for cid, g in merged.groupby('client_id_x', sort=False):
    freq_dict = compute_freq_vectorized(g['date'], window_days)
    for w in window_days:
        merged.loc[g.index, f'TxnFrequency_{w}d'] = freq_dict[w]

# --- AmtDelta ---
merged['prev_amount'] = merged.groupby('client_id_x')['amount'].shift(1)
merged['AmtDelta'] = merged['amount'] - merged['prev_amount']
merged['AmtDelta'] = merged['AmtDelta'].fillna(0)
merged.drop(columns='prev_amount', inplace=True)

In [10]:
# US region mapping
us_region_map = {
    'Northeast': ['NY','NJ','PA','MA','CT','RI','NH','VT','ME'],
    'Midwest': ['IL','OH','MI','IN','WI','MN','IA','MO','ND','SD','NE','KS'],
    'South': ['FL','GA','SC','NC','AL','MS','LA','TX','OK','TN','KY','VA','WV','AR','MD','DE','DC'],
    'West': ['CA','WA','OR','NV','AZ','NM','CO','UT','ID','MT','WY','AK','HI'],
}
continent_map = {
    'Europe': [ ... ],  # 原本 continent_map['Europe'] 可直接使用
    'Online': ['online','AA']
}

us_region_lookup = {state: region for region, states in us_region_map.items() for state in states}

# --- 向量化 location 特徵 ---
merged['merchant_online'] = merged['merchant_state'].eq('online').astype('uint8')
merged['merchant_us'] = merged['merchant_state'].isin(us_region_lookup.keys()).astype('uint8')
merged['merchant_eu'] = merged['merchant_state'].isin(continent_map['Europe']).astype('uint8')
merged['merchant_others'] = (~merged[['merchant_online','merchant_us','merchant_eu']].any(axis=1)).astype('uint8')


In [11]:
# --- 首次交易標記 ---
merged['FirstTxnInRegion'] = (~merged.duplicated(subset=['client_id_x', 'merchant_state'])).astype('uint8')

In [12]:
#merged[["card_id","card_number"]]
import numpy as np
from scipy import stats 

# === (1) log轉換 ===
merged['amount'] = np.where(merged['amount'] < 0, 0, merged['amount'])  # 負數變 0
merged['amount'] = np.log(merged['amount'] + 1)  

# === (3) 平方根轉換 ===
merged['credit_limit']=np.sqrt(merged['credit_limit'])
merged['total_debt']=np.sqrt(merged['total_debt'])

# === (3) 立方根轉換 ===
merged['yearly_income']=np.cbrt(merged['yearly_income'])
merged['per_capita_income']=np.cbrt(merged['per_capita_income'])

## Box-Cox Transformation
###merged['yearly_income'], fitted_lambda = stats.boxcox(merged['yearly_income'])

# === (5) Yeo–Johnson 轉換（可處理負值） ===
###merged['per_capita_income'], lambdaValue =stats.yeojohnson(merged['per_capita_income'])

## 04-2_分割訓練集及測試集

In [13]:
# --- 選取數值型變數 ---
num_cols = merged.select_dtypes(include=['int64', 'float64','uint8','datetime64[ns]']).columns
df2 = merged[num_cols]

# --- dropna ---
df_cleaned = df2.dropna()
del df2

# --- 避免共線性 ---
df_cleaned.drop(columns=["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction"], inplace=True)

# --- 確保 date 欄位在 df_cleaned 中 ---
if 'date' not in df_cleaned.columns:
    df_cleaned['date'] = merged.loc[df_cleaned.index, 'date']

# --- 依時間排序 ---
df_sorted = df_cleaned.sort_values('date')

# --- 時間序列切分（前 80% 訓練, 後 20% 測試） ---
split_index = int(len(df_sorted) * 0.8)
train_df = df_sorted.iloc[:split_index].drop(columns=['date'])  # 可選擇丟掉 date
test_df  = df_sorted.iloc[split_index:].drop(columns=['date'])

# --- 檢查詐欺資料分布 ---
print(train_df['is_fraud'].value_counts(normalize=False))
print(test_df['is_fraud'].value_counts(normalize=False))

# --- 清理不用的變數 ---
del df_cleaned, df_sorted, merged

is_fraud
0    1367157
1       2393
Name: count, dtype: Int64
is_fraud
0    341792
1       596
Name: count, dtype: Int64


In [ ]:
# -----------------------------
# 1. 用 train_df 計算 fraud rate
# -----------------------------
fraud_rate_train = (
    train_df.groupby('mcc_code')['is_fraud'].mean()
)

# 設定 threshold = 2%
high_risk_MCC_list = fraud_rate_train[fraud_rate_train > 0.02].index.tolist()

# -----------------------------
# 2. 加入特徵到 train_df / test_df
# -----------------------------
train_df['HighRiskMCC'] = train_df['mcc_code'].isin(high_risk_MCC_list).astype('uint8')
test_df['HighRiskMCC']  = test_df['mcc_code'].isin(high_risk_MCC_list).astype('uint8')

In [ ]:
train_df.columns

In [14]:
cols_to_drop = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df  = test_df.drop(columns=cols_to_drop, errors='ignore')


## 04-3(b1)_(略04-3(a))Assumption:Avoid Multicollinearity

In [15]:
##處理共線性
train_df.drop(columns=["per_capita_income"], inplace=True)
train_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
train_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)
#再重跑一次VIF

In [16]:
test_df.drop(columns=["per_capita_income"], inplace=True)
test_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
test_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)

In [18]:
test_df.drop(columns=["mcc_code","errors_missing_flag"], inplace=True)
train_df.drop(columns=["mcc_code","errors_missing_flag"], inplace=True)

## 04-4 Elastic Net selection

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

def fast_elasticnet_feature_selection(train_df, dep_var="is_fraud", l1_ratio=0.5):

    y = train_df[dep_var]
    X = train_df.drop(columns=[dep_var])

    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    X = X[numeric_cols]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = SGDClassifier(
        loss="log_loss",
        penalty="elasticnet",
        alpha=0.0001,
        l1_ratio=l1_ratio,
        max_iter=20,
        n_jobs=-1
    )

    model.fit(X_scaled, y)

    coef = model.coef_.ravel()
    selected = [c for c, v in zip(numeric_cols, coef) if abs(v) > 1e-6]

    return selected

## 04-3 all丟模型內

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=1000,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [ ]:
selected_features = ['transaction_id', 'client_id_x', 'card_id', 'amount', 'merchant_id', 'zip', 'mcc_code', 'errors_missing_flag', 'use_chip_Swipe Transaction', 'current_age', 'retirement_age', 'latitude', 'longitude', 'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards', 'gender_Male', 'card_number', 'cvv', 'num_cards_issued', 'credit_limit', 'card_type_Debit', 'card_brand_Mastercard', 'has_chip_YES']

dep_var = "is_fraud"

# ⬇ 確保模型只吃 selected features + label
train_selected = train_df[selected_features + [dep_var]].copy()
test_selected = test_df[selected_features + [dep_var]].copy()


In [20]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

print(result["train_auc"],result["test_auc"],result["train_prauc"],result["test_prauc"])

Train AUC: 0.9955355101224314
Test AUC: 0.9980093519336537
Train PR-AUC: 0.5045248555573588
Test PR-AUC: 0.39802744296522374
0.9955355101224314 0.9980093519336537 0.5045248555573588 0.39802744296522374


## 04-4(c)_Forward/Backward v2

# 05_XGB

In [ ]:
# =========================
# 0. 套件
# =========================
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

# =========================
# 1. 分割 X, y
# =========================
X_train = train_df.drop(columns=["is_fraud"])
y_train = train_df["is_fraud"]

X_test = test_df.drop(columns=["is_fraud"])
y_test = test_df["is_fraud"]

# =========================
# 2. 對齊欄位 (train 為主)
# =========================
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# =========================
# 3. 處理類別不平衡
# =========================
ratio = (y_train == 0).sum() / (y_train == 1).sum()
print("scale_pos_weight =", ratio)

# =========================
# 4. 建立模型
# =========================
model = XGBClassifier(
    # ---- Core ----
    n_estimators=2000,             # 多樹 + 小 learning rate → 一般比較穩
    learning_rate=0.02,            # 再降低一點，讓 regularization 有效果
    objective='binary:logistic',
    eval_metric='auc',

    # ---- Complexity control ----
    max_depth=4,                   # 6 → 過深，fraud detection 通常 3–5 最穩
    min_child_weight=5,            # default=1 很容易 overfit
    gamma=1,                       # 要求 split 需有 gain → 減少雜訊
    subsample=0.8,
    colsample_bytree=0.8,

    # ---- Regularization ----
    reg_alpha=1,                   # L1，幫助稀疏化
    reg_lambda=2,                  # L2，控制 model variance

    # ---- Imbalance ----
    scale_pos_weight=ratio,        # 你原本的作法正確

    # ---- Infrastructure ----
    tree_method='hist',
    n_jobs=-1,
    use_label_encoder=False
)


# =========================
# 5. 訓練模型（early stopping）
# =========================
# 注意：XGBoost 2.x 用 callbacks 取代早期的 early_stopping_rounds 參數
from xgboost import callback

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    #early_stopping_rounds=50,
    verbose=50
)

# =========================
# 6. 預測
# =========================
pred_prob = model.predict_proba(X_test)[:, 1]

# =========================
# 7. 計算評估指標
# =========================
auc_score = roc_auc_score(y_test, pred_prob)
prauc_score = average_precision_score(y_test, pred_prob)

print("ROC AUC:", auc_score)
print("PR AUC:", prauc_score)


In [ ]:
pred_prob = model.predict_proba(X_train)[:, 1]
auc_score = roc_auc_score(y_train, pred_prob)
prauc_score = average_precision_score(y_train, pred_prob)

print("ROC AUC:", auc_score)
print("PR AUC:", prauc_score)


In [ ]:
# =========================
# 0. 套件
# =========================
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

# =========================
# 1. 分割 X, y
# =========================
X_train = train_df.drop(columns=["is_fraud"])
y_train = train_df["is_fraud"]

X_test = test_df.drop(columns=["is_fraud"])
y_test = test_df["is_fraud"]

# =========================
# 2. 對齊欄位 (train 為主)
# =========================
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# =========================
# 3. 處理類別不平衡
# =========================
ratio = (y_train == 0).sum() / (y_train == 1).sum()
print("scale_pos_weight =", ratio)

# =========================
# 4. 建立模型
# =========================
model = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.02,
    objective='binary:logistic',
    eval_metric='aucpr',  # PR-AUC 對不平衡資料更合理
    max_depth=3,           # 或 3~4 試比較
    min_child_weight=3,    # 或 3~5 試比較
    gamma=0.5,             # 或 0.5~1
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1,
    reg_lambda=2,
    scale_pos_weight=568,  # 根據正負樣本比例
    tree_method='hist',
    n_jobs=-1,
    use_label_encoder=False
)



# =========================
# 5. 訓練模型（early stopping）
# =========================
# 注意：XGBoost 2.x 用 callbacks 取代早期的 early_stopping_rounds 參數
from xgboost import callback

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=50,
    verbose=50
)

# =========================
# 6. 預測
# =========================
pred_prob = model.predict_proba(X_test)[:, 1]

# =========================
# 7. 計算評估指標
# =========================
auc_score = roc_auc_score(y_test, pred_prob)
prauc_score = average_precision_score(y_test, pred_prob)

print("ROC AUC:", auc_score)
print("PR AUC:", prauc_score)


In [ ]:
pred_prob = model.predict_proba(X_train)[:, 1]
auc_score = roc_auc_score(y_train, pred_prob)
prauc_score = average_precision_score(y_train, pred_prob)

print("ROC AUC:", auc_score)
print("PR AUC:", prauc_score)


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score

# 初始化結果表
result = pd.DataFrame(columns=["Model", "FeatureSet", "Train AUC", "Test AUC", "Train PR", "Test PR"])

# 函數：訓練模型並計算指標
def run_models(train_df, test_df, label_col="is_fraud", feature_name="AllFeatures"):
    # 特徵自動抓除了 label 之外的所有欄位
    feature_list = [c for c in train_df.columns if c != label_col]

    X_train = train_df[feature_list]
    y_train = train_df[label_col]

    X_test = test_df[feature_list]
    y_test = test_df[label_col]

    # -------- XGBoost --------
    xgb_model = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        tree_method="hist",
        enable_categorical=True,
        use_label_encoder=False
    )
    xgb_model.fit(X_train, y_train)

    train_pred = xgb_model.predict_proba(X_train)[:,1]
    test_pred  = xgb_model.predict_proba(X_test)[:,1]

    result.loc[len(result)] = [
        "XGBoost",
        feature_name,
        roc_auc_score(y_train, train_pred),
        roc_auc_score(y_test, test_pred),
        average_precision_score(y_train, train_pred),
        average_precision_score(y_test, test_pred)
    ]

    # -------- LightGBM --------
    lgb_model = lgb.LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8
    )
    lgb_model.fit(X_train, y_train)

    train_pred = lgb_model.predict_proba(X_train)[:,1]
    test_pred  = lgb_model.predict_proba(X_test)[:,1]

    result.loc[len(result)] = [
        "LightGBM",
        feature_name,
        roc_auc_score(y_train, train_pred),
        roc_auc_score(y_test, test_pred),
        average_precision_score(y_train, train_pred),
        average_precision_score(y_test, test_pred)
    ]

    # 印出結果
    print(result)
    return result

# 直接跑
result = run_models(train_df, test_df)


In [ ]:
run_xgb

In [ ]:
import pandas as pd

# 建立結果 DataFrame
result = pd.DataFrame(columns=[
    "model", "feature_set",
    "train_auc", "test_auc",
    "train_pr", "test_pr"
])

# 印出完整結果
print(result)

# 或者漂亮一點，用 DataFrame 的 style
display(result)

# 如果只想看某幾欄
print(result[["model", "test_auc", "test_pr"]])


In [ ]:
import sys
print(sys.executable)
import xgboost
print(xgboost.__file__)
print(xgboost.__version__)


# 99_其他code

In [ ]:
merged.to_csv("merged.csv", index=False)

In [ ]:
print("Train positive rate:", y_train.mean())
print("Test positive rate:", y_test.mean())


In [ ]:
from scipy.stats import ks_2samp

def ks_check(train_df, test_df):
    diffs = {}
    for col in train_df.columns:
        if train_df[col].dtype != 'O':   # numeric only
            stat, _ = ks_2samp(train_df[col], test_df[col])
            diffs[col] = stat
    return pd.Series(diffs).sort_values(ascending=False)

ks_result = ks_check(X_train, X_test)
print(ks_result.head(20))


In [ ]:
from scipy.stats import ks_2samp

for col in X_train.columns:
    stat, p = ks_2samp(train_df[col], test_df[col])
    print(col, "KS p-value:", p)
